# Yeast cells detection evaluation notebook

In [16]:
import sys
sys.path.append('../')
#sys.path.append('%HOMEPATH%/detectron2') #insert local path to detectron2
import numpy as np

import yeastcells.clustering as clustering
import yeastcells.features as features
import yeastcells.data as data
import yeastcells.model as model
import yeastcells.visualize as visualize
import yeastcells.evaluation as evaluation
%matplotlib notebook

In [17]:
cd ~

/home/p301081


### Load data

In [18]:
path = '../../slow-data/yeastcells/data/cellstar_benchmark/TestSet1'
filenames = data.load_data(path)

In [19]:
image = data.read_images_cat(filenames)
np.shape(image)

(60, 512, 512, 3)

### Load and run model on data for segmentation

In [20]:
#set path to model_final.pth
model_path = '/var/tensorflow-logs/p253591/yeast-cell-detection-run-4'

#load model
predictor = model.load_model(model_path, seg_thresh=0.955, device='cuda:0') #set GPU if available otherwise use 'cpu'

In [21]:
output = [
  {'instances': predictor(frame)['instances'].to('cpu')}
  for frame in image
]

### Run clustering algorithm for tracking

In [22]:
labels, coordinates = clustering.cluster_cells(output, dmax=5, min_samples=3, eps=0.6, progress=False)

/home/p301081/.virtualenvs/yeastcells/lib/python3.7/site-packages/sklearn/neighbors/_base.py:168: EfficiencyWarning: Precomputed sparse input was not sorted by data.
  EfficiencyWarning)


### Load ground truth data and reformat prediction outputs into array

In [23]:
seg_path = f'{path}/GroundTruth_Segmentation.csv'
track_path = f'{path}/GroundTruth_Tracking.csv'
gt_s, gt_t = data.get_gt_yit(seg_path, track_path)
pred_s, pred_t, pred_df = data.get_pred(output, labels, coordinates)

### Match predictions to ground truth

In [25]:
results_s = evaluation.get_seg_performance(pred_s, gt_s, output, pipeline='maskrcnn')
results_t = evaluation.get_track_performance(pred_t, gt_t, output, pipeline='maskrcnn')

In [26]:
print(results_s)
print(results_t)

{'tp': 1069, 'fp': 101, 'fn': 97, 'join': 0, 'split': 0}
{'tp': 1046, 'fp': 96, 'fn': 93, 'join': 0, 'split': 0}


In [27]:
metrics_s = evaluation.calculate_metrics(results_s, pred_s, gt_s)
metrics_t = evaluation.calculate_metrics(results_t, pred_t, gt_t)

In [28]:
print(metrics_s)
print(metrics_t)

{'F1-score': 0.9152397260273973, 'Accuracy': 0.8437253354380426, 'Precision': 0.9136752136752136, 'Recall': 0.9168096054888508}
{'F1-score': 0.9171416045594037, 'Accuracy': 0.8469635627530364, 'Precision': 0.9159369527145359, 'Recall': 0.9183494293239683}
